In [1]:
# Init
import os
import sys

module_path = os.path.abspath(os.path.join("../src/simulicronalpha/"))
if module_path not in sys.path:
    sys.path.append(module_path)

# Imports
import numpy as np
import pandas as pd
import warnings
import multiprocessing
np.set_printoptions(suppress=True)
from numpy import concatenate as c
from numpy import cumsum
import random
%load_ext line_profiler

In [2]:
from generateSim import (generatePopulation, generateGenome)
from stats import stats
from regulation import regulation
from checkCopyNumber import checkCopyNumber
#from recombination import recombination

In [3]:
from popSim import runBatch, runSim

In [4]:
gen,piset,piIndice, rates = generateGenome(
    numberOfInsertionSites=10000,
    numberOfChromosomes=6,
    baseRecombinationRate=0.1,
    baseSelection=0,
)
pop, tr , TEset= generatePopulation(
    gen,
    piIndice,
    NumberOfIndividual=1000,
    NumberOfTransposonTypes=1,
    NumberOfInsertionsPerType=[1],
    FrequencyOfInsertions=[1.0],
    ExcisionRates = [1.0]
)

In [7]:
%lprun -f runSim runSim(gen,pop,tr,TEset,1,30,rates)

Timer unit: 1e-06 s

Total time: 279.298 s
File: /home/siddharth/Documents/Projects/Simulicron/src/simulicronalpha/popSim.py
Function: runSim at line 25

Line #      Hits         Time  Per Hit   % Time  Line Contents
    25                                           def runSim(
    26                                               genomeMatrix,
    27                                               populationMatrix,
    28                                               transposonMatrix,
    29                                               TEset,
    30                                               NumberOfTransposonInsertions,
    31                                               generations,
    32                                               genMap,
    33                                           ):
    34                                               # ------------------#
    35                                               # lambda/macros
    36         1          2.0      2.0      0.0

In [ ]:
%lprun -f recombination recombination(rates=rates, transposonMatrix=tr, v1=[52], v2=[65])

In [ ]:
%lprun -f runBatch runBatch(numberOfSimulations=1,baseSelection=0,baseInsertionProb=1,numberOfInsertionSites=10000,numberOfChromosomes=6,baseRecombinationRate=0.01,baseTau=1,numberOfPiRNA=6,piPercentage=3,enablePiRecombination=False,NumberOfIndividual=1000,NumberOfTransposonTypes=1,NumberOfInsertionsPerType=[1],FrequencyOfInsertions=[1.0],ExcisionRates=[1.0],RepairRates=[1],InsertionRates=[1],HardyWeinberg=False,NumberOfGenerations=10,numberOfThreads=1,)

In [ ]:
recombination(rates=rates, transposonMatrix=tr, v1=[52,65], v2=[52,65])

In [ ]:
np.set_printoptions(suppress=True,)

In [ ]:
def recombination(rates, transposonMatrix, v1, v2):
    # Empty vectors to store result
    r1 = []
    r2 = []
    # Creating lambda (macro)
    # "Match" does not exist in python
    # match = lambda a, b: [ b.index(x) if x in b else 0 for x in a ]
    # Get the postion of transposons 
    positionV1 = transposonMatrix[v1,1].astype(int).tolist()
    positionV2 = transposonMatrix[v2,1].astype(int).tolist()
    # This step sorts the locations and adds another location, 
    # 0 if not already present
    unqiquePos = list(set(positionV1+positionV2) | set([0]))
    unqiquePos.sort()
    # Calculate the effective rate from genome map
    effectiveRates = 0.5*(1-np.exp(-2*np.diff(rates[unqiquePos])))
    # print (effectiveRates)
    # Performing "Recombination"
    rec = (np.random.uniform(size=len(effectiveRates)) < effectiveRates)
    # Select the direction to start from
    start = [(np.random.uniform() < 0.5)]
    # Concat. start and recombination
    # Also remove the added 0 and start from
    # whichhaplo and uniqpos
    whichhaplo = 1 + cumsum(c((start, rec))) % 2
    whichhaplo = np.delete(whichhaplo, 0)
    del unqiquePos[0]
    unqiquePos = np.asarray(unqiquePos)
    # Generating the haplotype
    # Also checking if there is no transposon left
    # In the case above, return a (int) 0
    # Else return the array containing transposons
    if (positionV1 == [0]):
        r1 = []
    else:
        if not any(whichhaplo == 1):
            pass
        else:
            pos = unqiquePos[whichhaplo == 1]
            for i in v1:
                if (int(transposonMatrix[i,1]) in pos):
                    r1.append(i)
    if (positionV2 == [0]):
        r2 = []
    else:
        if not any(whichhaplo == 2):
            pass
        else:
            pos = unqiquePos[whichhaplo == 2]
            for i in v2:
                if (int(transposonMatrix[i,1]) in pos):
                    r2.append(i)
    # Merge to create gamate
    r = r1 + r2
    # Return 0 if no transposon remains
    if not r:
        return (0)
    else:
        return r